In [ ]:
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Dense, Dropout, SimpleRNN, RepeatVector
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback

from termcolor import colored

In [ ]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

import pandas as pd
import matplotlib.pyplot as plt

wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1ot8kUHRmmquThfm4Fu8FGHVueDG0HQvgAl7DC4PyNEk/edit#gid=854215923')

In [ ]:
sheet = wb.worksheet('predicted')
data = sheet.get_all_values()

In [ ]:
df = pd.DataFrame(data)
df.columns = df.iloc[0]
df = df.iloc[1:]

In [ ]:
df.head()

,0,Captions,Grammar
1,1003163366_44323f5815.jpg,ایک قمیض کے بغیر شخص اپنے کتے کے ساتھ پارک بین...,1
2,1003163366_44323f5815.jpg,ایک شخص بینچ پر لیٹا ہے جبکہ اس کا کتا اس کے س...,1
3,1003163366_44323f5815.jpg,ایک شخص بینچ پر لیٹا ہے جبکہ اس کے ساتھ اپنے ک...,0
4,1003163366_44323f5815.jpg,ایک شخص بینچ پر لیٹا ہے جبکہ دوسرا شخص بینچ پر...,1
5,1003163366_44323f5815.jpg,ایک قمیض کے بغیر شخص اپنے کتے کے ساتھ پارک بین...,1


In [ ]:
ind = 1005

In [ ]:
labels = df.Grammar.tolist()
labels = labels[:ind]
captions = df.Captions.tolist()
captions = captions[:ind]

In [ ]:
max_len = max([len(caption) for caption in captions])
max_len

251

In [ ]:
chars = {'<','e','n','d','>'}
for i in range(len(captions)):
    caption = captions[i]
    caption = caption.replace('\u200b','')
    caption = caption.replace('\u200d','')
    captions[i] = caption
    cap = set(list(caption))
    chars = chars.union(cap)

chars = sorted(list(chars))
len(chars)

59

In [ ]:
all_chars='1'
for char in chars:
    all_chars = all_chars + char

all_chars

'1 02578<>adentvآؤئابتثجحخدذرزسشصضطظعغفقلمنهوُِٹپچڈڑژکگںھہیےۓ'

In [ ]:
num_features = len(all_chars)
print('number of features:', num_features)
char_to_index = dict((c,i) for i,c in enumerate(all_chars))
index_to_char = dict((i,c) for i,c in enumerate(all_chars))

number of features: 60


In [ ]:
def generate_data():
    return captions,labels

## Task4: Vectorize and De-Vectorize Data

In [ ]:
def vectorize_example(example,label):
    x = np.zeros((max_len, num_features))
    # y = np.zeros((max_time_steps, num_features))

    diff_x = max_len - len(example)
    # diff_y = max_time_steps - len(label)

    for i,c in enumerate(example):
        x[i+diff_x, char_to_index[c]] = 1
    for i in range(diff_x):
        x[i,char_to_index[' ']] = 1
    # for i,c in enumerate(label):
    #     y[i+diff_y, char_to_index[c]] = 1
    # for i in range(diff_y):
    #     y[i,char_to_index[' ']] = 1
    return x,np.array(label).reshape((1))

e,l = generate_data()
print(e[2],l[2])

x,y = vectorize_example(e[1],l[1])
print(x.shape,y.shape)

ایک شخص بینچ پر لیٹا ہے جبکہ اس کے ساتھ اپنے کتے کے ساتھ پارک بینچ پر پڑا ہے <end> 0
(251, 60) (1,)


In [ ]:
def devectorize_example(example):
    if len(example) == 1:
        return str(int(example))
    result = [index_to_char[np.argmax(vec)] for i,vec in enumerate(example)]
    return ''.join(result)

devectorize_example(x)

'                                                                                                                                                                                ایک شخص بینچ پر لیٹا ہے جبکہ اس کا کتا اس کے ساتھ پارک بینچ پر پڑا ہے <end>'

In [ ]:
devectorize_example(y)

'1'

## Task5: Create Dataset

In [ ]:
def create_dataset(num_examples):
    x = np.zeros((num_examples,max_len, num_features))
    y = np.zeros((num_examples,1))

    for i in range(num_examples):
        e,l = captions[i], labels[i]
        e_v, l_v = vectorize_example(e,l)
        #print(i,l,l_v)
        x[i] = e_v
        y[i] = l_v

    return x,y

x,y = create_dataset(ind)    
print(x.shape, y.shape)

(1005, 251, 60) (1005, 1)


In [ ]:
devectorize_example(x[2])

'                                                                                                                                                                         ایک شخص بینچ پر لیٹا ہے جبکہ اس کے ساتھ اپنے کتے کے ساتھ پارک بینچ پر پڑا ہے <end>'

In [ ]:
devectorize_example(y[2])

'0'

In [ ]:
rand_i = list(range(0,len(y)))

tr_p = int(len(rand_i)*0.6)
vl_p = int(len(rand_i)*0.8)
trainX = np.array([x[i] for i in rand_i[:tr_p]])
trainY = np.array([y[i] for i in rand_i[:tr_p]])
validX = np.array([x[i] for i in rand_i[tr_p:vl_p]])
validY = np.array([y[i] for i in rand_i[tr_p:vl_p]])
testX = np.array([x[i] for i in rand_i[vl_p:]])
testY = np.array([y[i] for i in rand_i[vl_p:]])

print("trainX.shape",trainX.shape)
print("trainY.shape",trainY.shape)
print("validX.shape",validX.shape)
print("validY.shape",validY.shape)
print("testX.shape",testX.shape)
print("testY.shape",testY.shape)


trainX.shape (603, 251, 60)
trainY.shape (603, 1)
validX.shape (201, 251, 60)
validY.shape (201, 1)
testX.shape (201, 251, 60)
testY.shape (201, 1)


## Task6: Training the Model

In [ ]:
classifier = Sequential()
#First Hidden Layer
classifier.add(Dense(251*60, activation='relu', kernel_initializer='random_normal'))
#Second  Hidden Layer
classifier.add(Dense(1024, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dense(128, activation='relu', kernel_initializer='random_normal'))
#Output Layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

In [ ]:
classifier.fit(x,y, batch_size=50, epochs=50)

Epoch 1/50
21/21 [==============================] - 402s 19s/step - loss: 0.6724 - accuracy: 0.6224
Epoch 2/50
21/21 [==============================] - 403s 19s/step - loss: 0.6622 - accuracy: 0.6337
Epoch 3/50
21/21 [==============================] - 400s 19s/step - loss: 0.6612 - accuracy: 0.6338
Epoch 4/50
21/21 [==============================] - 401s 19s/step - loss: 0.6594 - accuracy: 0.6338
Epoch 5/50
21/21 [==============================] - 401s 19s/step - loss: 0.6556 - accuracy: 0.6338
Epoch 6/50
21/21 [==============================] - 402s 19s/step - loss: 0.6558 - accuracy: 0.6338
Epoch 7/50
21/21 [==============================] - 403s 19s/step - loss: 0.6583 - accuracy: 0.6338
Epoch 8/50
21/21 [==============================] - 403s 19s/step - loss: 0.6581 - accuracy: 0.6338
Epoch 9/50
21/21 [==============================] - 402s 19s/step - loss: 0.6562 - accuracy: 0.6338
Epoch 10/50
21/21 [==============================] - 402s 19s/step - loss: 0.6567 - accuracy: 0.6338